In [12]:
pip install transformers torch nltk flask

In [20]:
import pandas as pd
from transformers import pipeline

def load_data():
    #dataset
    data = pd.read_csv('conversations.csv')
    return data

def initialize_ner_model():
    #pre-trained NER model
    ner_model = pipeline("ner", aggregation_strategy="simple")
    return ner_model

data = load_data()
ner_model = initialize_ner_model()

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-p

In [26]:
import random

responses = {
    "greetings": ["Hello! How can I assist you today?", "Hi there! What can I do for you?", "Hey! How can I help?"],
    "goodbye": ["Goodbye! Have a great day!", "See you later!", "Take care!"],
    "flight_booking": ["Where would you like to fly?", "I'll book your flight right away!"],
    "weather": ["The weather is sunny today.", "It's raining in some areas.", "Expect cloudy skies today."],
    "cancel_booking": ["Your booking has been canceled.", "I've successfully canceled your booking."],
    "reminder": ["Reminder set!", "I'll remind you at the specified time."],
    "flight_status": ["Let me check your flight status.", "Your flight is on time.", "Your flight is delayed."]
}

#closest intent
def find_intent(user_input):
    for _, row in data.iterrows():
        if row['user_input'].lower() in user_input.lower():
            return row['intent']
    return "unknown"

#chatbot response
def simulate_response(user_input):
    intent = find_intent(user_input)

    if intent == "unknown":
        return "I'm sorry, I didn't understand that.", []
    response = random.choice(responses[intent])

    entities = ner_model(user_input)
    return response, entities

In [28]:
def chatbot():
    print("Chatbot: Hello! I am here to help you. (Type 'exit' to end the conversation.)")
    conversation_feedback = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break

        #get response, entities, and intent
        response, entities = simulate_response(user_input)
        intent = find_intent(user_input)  # Get the intent

        #chatbot response, entities, and intent
        print(f"Chatbot: {response}")
        print(f"Intent: {intent}")  # Print the intent
        if entities:
            print(f"Entities: {entities}")
        else:
            print("No entities found.")

        #feedback
        feedback = input("Please rate my response (1-5): ")
        conversation_feedback.append({
            'user_input': user_input,
            'intent': intent,
            'entities': entities,
            'feedback': feedback
        })

        if int(feedback) <= 2:
            print("Thank you for your feedback! I'll try to improve next time.")
        else:
            print("Thank you! I'm glad you found it helpful.")

if __name__ == "__main__":
    chatbot()

Chatbot: Hello! I am here to help you. (Type 'exit' to end the conversation.)
You: Cancel my flight booking to London
Chatbot: Your booking has been canceled.
Intent: cancel_booking
Entities: [{'entity_group': 'LOC', 'score': 0.9986123, 'word': 'London', 'start': 28, 'end': 34}]
Please rate my response (1-5): 5
Thank you! I'm glad you found it helpful.
You: exit
